In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import KSP_module as ksp
from KSP_module import Kerbol, Kerbin, Duna, Eve, Mun, Minmus

HKO = ksp.orbit(Kerbin, min_alt = 70000.1, e=0.99)
t_leave = HKO.calc_soi_leave_time(0)

fig,ax = ksp.initialize_plot()
ax = ksp.add_soi_to_plot(ax, Kerbin, 0, True, True)
ax = ksp.add_orbit_to_plot(ax, HKO)
ax = ksp.add_orbit_point_to_plot(ax, HKO, t_leave)
print(HKO.primary.soi - HKO.calc_polar(t_leave)[0])

In [ ]:
LKO = ksp.orbit(Kerbin, min_alt = 70000.1, e=0)
transfer3 = ksp.calc_window(LKO, Mun.orbit, 0)
fig,ax = ksp.plot_hohmann_orbit(LKO, Mun, transfer3)

t_soi, new_primary = LKO.calc_soi_change(transfer3.t_launch)

print(transfer3.t_launch)
ax = ksp.add_soi_to_plot(ax, Mun, t_soi)
ax = ksp.add_planetary_body_to_plot(ax, Mun, transfer3.t_launch)
ax = ksp.add_planetary_body_to_plot(ax, Kerbin, transfer3.t_launch, True)
plt.show()
print([ksp.pretty_date(transfer3.t_launch), transfer3.t_launch])

In [1]:
import KSP_module as ksp
from KSP_module import Kerbol, Kerbin, Duna, Eve, Mun, Minmus, Moho, Jool, Dres
transfer = ksp.calc_window(Kerbin.orbit, Duna.orbit, 0)

KSP module loaded
All tests passed


In [3]:
transfer.dst_body

Kerbol

In [35]:
from math import sqrt

def calc_hohmann_dv(src_orbit, dst_orbit):
    """Calculate delta-v for Hohmann escape or capture, eg. from/to LKO"""
    
    # either source or destination must be a Hohmann orbit (exactly one)
    assert(src_orbit.is_hohmann ^ dst_orbit.is_hohmann), 'Exactly one orbit must be a Hohmann orbit'
    # chack which one is which
    if src_orbit.is_hohmann:
        hohmann_orbit = src_orbit
        parking_orbit = dst_orbit
        is_capture = True
        # this is to check that the destination and the Hohmann orbit are at the same position
        # at the time of arrival, and calculate the dv
        t_calc = hohmann_orbit.t_arrival
    else:
        hohmann_orbit = dst_orbit
        parking_orbit = src_orbit
        is_capture = False
        # this is to check that the source and the Hohmann orbit are at the same position
        # at the time of launch, and calculate the dv
        t_calc = hohmann_orbit.t_launch

    # parking is around a body orbiting the same primary as the Hohmann orbit
    parking_body = parking_orbit.primary
    assert(parking_body.orbit.primary == hohmann_orbit.primary),\
        'Parking orbit must be around a body orbiting the same primary as the Hohmann orbit'
    
    # check that the two orbits are plausible
    parking_p0 = parking_body.orbit.calc_xyz(t_calc)
    hohmann_p0 = hohmann_orbit.calc_xyz(t_calc)
    # np.testing.assert_almost_equal(parking_p0[:2], hohmann_p0[:2], decimal=10, err_msg='', verbose=True),\
    #     'Parking orbit and Hohmann orbit must be at similar x,y position either at launch or arrival'
    
    # calculate dv to/from Hohmann orbit, relative to the primary body of the parking orbit
    # this is the speed when leaving/entering the SOI
    v_soi = abs(hohmann_orbit.calc_speed(t_calc) - parking_body.orbit.calc_speed(t_calc))
    r_soi = parking_body.soi

    v_parking = parking_orbit.calc_speed(t_calc)
    r_parking = parking_orbit.a
    
    # calculate speed at parking orbit height using vis-viva equation
    GM = parking_body.GM
    v_hohmann_at_parking = sqrt(v_soi**2 + 2*GM/r_parking - 2*GM/r_soi)
    return v_hohmann_at_parking - v_parking

print(calc_hohmann_dv(LKO, transfer))
LDO = ksp.orbit(Duna, min_alt = Duna.atmo_height+0.1, e=0)
print(calc_hohmann_dv(transfer, LDO))

calc_hohmann_dv(LKO, LKO)

1052.282848202633
643.2574722531166


AssertionError: Exactly one orbit must be a Hohmann orbit

In [8]:
import numpy as np